In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('E:\Me\Health survey.xlsx', engine='openpyxl')

In [3]:
df.columns

Index(['Timestamp', 'ID', 'Age', 'Gender', 'Country', 'state', 'self_employed',
       'family_history', 'treatment', 'work_interfere', 'no_employees',
       'remote_work', 'tech_company', 'benefits', 'care_options',
       'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments'],
      dtype='object')

In [4]:
df_mod = df.copy()

In [5]:
df_mod = df_mod.drop(columns='Timestamp')

In [6]:
id_vars = list(df_mod.columns)[:5]
id_vars

['ID', 'Age', 'Gender', 'Country', 'state']

In [7]:
id_vars = list(df_mod.columns)[:5]
value_vars = list(df_mod.columns)[5:]

In [8]:
df_mod_melted = df_mod.melt(id_vars=id_vars, value_vars=value_vars, var_name='Questions', value_name='Answers')
df_mod_melted

,ID,Age,Gender,Country,state,Questions,Answers
0,1.0,37.0,Female,United States,IL,self_employed,NaN
1,2.0,44.0,M,United States,IN,self_employed,NaN
2,3.0,32.0,Male,Canada,NaN,self_employed,NaN
3,4.0,31.0,Male,United Kingdom,NaN,self_employed,NaN
4,5.0,31.0,Male,United States,TX,self_employed,NaN
...,...,...,...,...,...,...,...
27693,NaN,NaN,NaN,NaN,NaN,comments,NaN
27694,NaN,NaN,NaN,NaN,NaN,comments,NaN
27695,NaN,NaN,NaN,NaN,NaN,comments,NaN
27696,NaN,NaN,NaN,NaN,NaN,comments,NaN


In [9]:
respondent = df_mod_melted[df_mod_melted['Answers'].notna()]
respondent =  respondent.groupby('Questions')['ID'].nunique().reset_index()
respondent.rename(columns={'ID':'Respondents'}, inplace=True)
respondent

,Questions,Respondents
0,anonymity,598
1,benefits,598
2,care_options,598
3,comments,77
4,coworkers,598
5,family_history,598
6,leave,598
7,mental_health_consequence,598
8,mental_health_interview,598
9,mental_vs_physical,598


In [10]:
df_merged = pd.merge(left=df_mod_melted, right=respondent, how='left', left_on="Questions", right_on="Questions")
print('Original data', len('df_mod_melted'))
print('Merged data', len('df_merged'))
print(df_merged)

Original data 13
Merged data 9
        ID   Age  Gender         Country state      Questions Answers  \
0      1.0  37.0  Female   United States    IL  self_employed     NaN   
1      2.0  44.0       M   United States    IN  self_employed     NaN   
2      3.0  32.0    Male          Canada   NaN  self_employed     NaN   
3      4.0  31.0    Male  United Kingdom   NaN  self_employed     NaN   
4      5.0  31.0    Male   United States    TX  self_employed     NaN   
...    ...   ...     ...             ...   ...            ...     ...   
27693  NaN   NaN     NaN             NaN   NaN       comments     NaN   
27694  NaN   NaN     NaN             NaN   NaN       comments     NaN   
27695  NaN   NaN     NaN             NaN   NaN       comments     NaN   
27696  NaN   NaN     NaN             NaN   NaN       comments     NaN   
27697  NaN   NaN     NaN             NaN   NaN       comments     NaN   

       Respondents  
0              580  
1              580  
2              580  
3       

In [11]:
df_merged['Answers']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
27693    NaN
27694    NaN
27695    NaN
27696    NaN
27697    NaN
Name: Answers, Length: 27698, dtype: object

In [12]:
same_answer = df_merged
#same_answer = same_answer.groupby(['Questions', 'Answers'])['Respondents'].nunique().reset_index()
same_answer = same_answer.groupby(['Questions', 'Answers'])['Respondents'].count().reset_index()
same_answer.rename(columns={'Respondents':'Same Answer'}, inplace=True)
same_answer

,Questions,Answers,Same Answer
0,anonymity,Don't know,401
1,anonymity,No,28
2,anonymity,Yes,169
3,benefits,Don't know,192
4,benefits,No,166
...,...,...,...
135,wellness_program,Yes,102
136,work_interfere,Never,110
137,work_interfere,Often,60
138,work_interfere,Rarely,88


In [13]:
df_merged_two = pd.merge(left=df_merged, right=same_answer, how='left', left_on=['Questions', 'Answers'], right_on=['Questions', 'Answers'])
df_merged_two['Same Answer'].fillna(0, inplace=True)
print('Original data', len('df_merged'))
print('Merged data', len('df_merged_two'))
print(df_merged_two)

Original data 9
Merged data 13
        ID   Age  Gender         Country state      Questions Answers  \
0      1.0  37.0  Female   United States    IL  self_employed     NaN   
1      2.0  44.0       M   United States    IN  self_employed     NaN   
2      3.0  32.0    Male          Canada   NaN  self_employed     NaN   
3      4.0  31.0    Male  United Kingdom   NaN  self_employed     NaN   
4      5.0  31.0    Male   United States    TX  self_employed     NaN   
...    ...   ...     ...             ...   ...            ...     ...   
27693  NaN   NaN     NaN             NaN   NaN       comments     NaN   
27694  NaN   NaN     NaN             NaN   NaN       comments     NaN   
27695  NaN   NaN     NaN             NaN   NaN       comments     NaN   
27696  NaN   NaN     NaN             NaN   NaN       comments     NaN   
27697  NaN   NaN     NaN             NaN   NaN       comments     NaN   

       Respondents  Same Answer  
0              580          0.0  
1              580      

In [14]:
Gender_value = df_merged_two['Gender'].unique()
Gender_value

array(['Female', 'M', 'Male', 'male', 'female', 'm', 'Male-ish', 'maile',
       'Trans-female', 'Cis Female', 'F', 'something kinda male?',
       'Cis Male', 'Woman', 'f', 'Mal', 'Male (CIS)', 'queer/she/they',
       'non-binary', 'Femake', 'woman', 'Make', 'Nah', 'All', 'Enby',
       'fluid', 'Genderqueer', 'Female ', 'Androgyne', 'Agender', nan],
      dtype=object)

In [15]:
df_merged_two['Gender'] = df_merged_two['Gender']. replace(['M', 'male', 'm', 'Male-ish', 'maile', 'Mal', 'Make'], 'Male')
df_merged_two['Gender'] = df_merged_two['Gender']. replace(['female', 'F', 'Woman', 'f', 'Femake', 'woman', 'Female '], 'Female')
df_merged_two['Gender'] = df_merged_two['Gender']. replace(['Trans-female', 'Cis Female', 'something kinda male?', 'Male (CIS)', 'queer/she/they','Cis Male', 'non-binary', 'Nah', 'All','Enby', 'fluid','Genderqueer', 'Androgyne','Agender'], 'Other')
Gender_value = df_merged_two['Gender'].unique()
Gender_value

array(['Female', 'Male', 'Other', nan], dtype=object)

In [16]:
df_merged_two['Age'].unique()


array([ 3.70e+01,  4.40e+01,  3.20e+01,  3.10e+01,  3.30e+01,  3.50e+01,
        3.90e+01,  4.20e+01,  2.30e+01,  2.90e+01,  3.60e+01,  2.70e+01,
        4.60e+01,  4.10e+01,  3.40e+01,  3.00e+01,  4.00e+01,  3.80e+01,
        5.00e+01,  2.40e+01,  1.80e+01,  2.80e+01,  2.60e+01,  2.20e+01,
        1.90e+01,  2.50e+01,  4.50e+01,  2.10e+01, -2.90e+01,  4.30e+01,
        5.60e+01,  6.00e+01,  5.40e+01,  3.29e+02,  5.50e+01,  1.00e+11,
        4.80e+01,  2.00e+01,  5.70e+01,  5.80e+01,  4.70e+01,  6.20e+01,
        5.10e+01,  6.50e+01,       nan])

In [17]:
print(df_merged_two['Age'].median())

30.0


In [18]:
df_merged_two["Age"] = np.where(df_merged_two["Age"] <0, 30,df_merged_two['Age'])
df_merged_two["Age"] = np.where(df_merged_two["Age"] >100, 30,df_merged_two['Age'])
df_merged_two['Age'].unique()

array([37., 44., 32., 31., 33., 35., 39., 42., 23., 29., 36., 27., 46.,
       41., 34., 30., 40., 38., 50., 24., 18., 28., 26., 22., 19., 25.,
       45., 21., 43., 56., 60., 54., 55., 48., 20., 57., 58., 47., 62.,
       51., 65., nan])

In [19]:
df_merged_two['Answers'].fillna("No answer", inplace=True)
df_merged_two['Age'].fillna(30, inplace=True)


In [20]:
df_merged_two.to_excel('Output.xlsx', sheet_name='Sheet1', index=False)